In [35]:
#!pip install rquge
#!pip install transformers
#!pip install sentencepiece
#!pip install git+https://github.com/alirezamshi/RQUGE
#!pip install evaluate
#!pip install sentence-transformers
#!pip install pandas
#!pip install --user sentencepiece
#!pip install git+https://github.com/alirezamshi/RQUGE
#!pip install rouge_score absl-py

In [1]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load
import pandas as pd
import sentencepiece as spm
from tqdm import tqdm
from datasets import load_metric



In [7]:
# with open('/Users/eraparihar/Desktop/Semester 2/SI 630/project/squad_train-v2.0.json', 'r') as file:
#     train_data = json.load(file)

converting the format for json file

In [8]:
# def convert_to_dataset(data):
#     dataset = []
#     for entry in data['data']:
#         title = entry['title']
#         for paragraph in entry['paragraphs']:
#             context = paragraph['context']
#             for qa in paragraph['qas']:
#                 q = qa['question']
#                 id_ = qa['id']
#                 is_impossible = qa['is_impossible']
#                 answers = qa['answers'] if not is_impossible else []
#                 # For each question, we will create a dictionary with the question, its context and the answers
#                 dataset.append({
#                     'id': id_,
#                     'title': title,
#                     'context': context,
#                     'question': q,
#                     'is_impossible': is_impossible,
#                     'answers': [{'text': answer['text'], 'answer_start': answer['answer_start']} for answer in answers]
#                 })
#     return dataset

In [2]:
with open('trial_data.json', 'r') as file:
    train_data = json.load(file)

In [2]:
#trial_datad = trial_data.to_dict(orient='records') # coverting df to dict
with open('trial_data.json', 'r') as file:
    trial_data = json.load(file)

In [3]:
data = trial_data[0:100]
len(data)

100

input model name below

In [4]:

model_name = "allenai/unifiedqa-t5-large" # can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Sample run_model, can try using different arguments with calling the function 
# run_model("which is best conductor? \\n (a) iron (b) feather (c) wood (d) plastic",
#          temperature=0.9, num_return_sequences=4, num_beams=20)

run_model("which is best conductor? \\n (a) iron (b) feather (c) wood (d) plastic",
         temperature=0.9)

In [6]:
generated_questions = []
contexts = []
generated_answers = []

In [7]:

for entry in tqdm(data, desc="Processing entries"):
    input_string = f"Question: {entry['question']} Context: {entry['context']}"
    output = run_model(input_string, )
    generated_answers.append(output[0])  
    contexts.append(entry['context'])
    generated_questions.append(entry['question'])


Processing entries: 100%|██████████| 100/100 [04:48<00:00,  2.89s/it]


In [8]:
generated_answers 

['late 1990s',
 'singing and dancing',
 '2003',
 'Houston, Texas',
 '1990s',
 "Destiny's Child",
 'Dangerously in Love',
 'her father, Mathew Knowles',
 'late 1990s',
 'lead singer',
 'Dangerously in Love',
 '2003',
 'five',
 'lead singer',
 'Dangerously in Love',
 'acting',
 'Jay Z',
 'six',
 'Dreamgirls',
 '2010',
 'Beyoncé',
 'Cadillac Records',
 'June 2005',
 "B'Day",
 'Dreamgirls',
 'Jay Z',
 'Sasha Fierce',
 'love, relationships, and monogamy',
 'influential',
 'Forbes',
 '2000s',
 'Forbes',
 'modern-day feminist',
 '2013 and 2014',
 '118 million',
 '60 million',
 '118 million',
 '20',
 'Forbes',
 "Destiny's Child",
 "her mother's maiden name",
 'African-American',
 'Methodist',
 'Xerox',
 'hairdresser',
 'Solange',
 'Joseph Broussard',
 'Xerox',
 'salon',
 'Solange',
 'Joseph Broussard',
 'Methodist',
 'Fredericksburg',
 'dance instructor Darlette Johnson',
 'Houston',
 'dance instructor Darlette Johnson',
 "St. John's United Methodist Church",
 'music magnet school',
 'Imagine'

In [10]:
rqugescore = load("alirezamsh/rquge")
results = rqugescore.compute(generated_questions=generated_questions, contexts=contexts, answers=generated_answers)
print("Mean RQUGE Score:", results["mean_score"])

Mean RQUGE Score: 4.91074542760849


In [18]:
# trial implementation for understanding

# from evaluate import load
# rqugescore = load("alirezamsh/rquge")
# generated_questions = ["how is the weather?"]
# contexts = ["the weather is sunny"]
# answers = ["sunny"]
# results = rqugescore.compute(generated_questions=generated_questions, contexts=contexts, answers=answers)
# print(results["mean_score"])

In [12]:
ground_truth_answers = [item['answers'][0]['text'] for item in data] 


In [13]:
# Evaluation metrics
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
rouge_metric = load_metric("rouge") 

# Calculate BLEU score
bleu_score = bleu_metric.compute(predictions=[generated_answers], references=[[ground_truth_answers]])

# Calculate METEOR score
meteor_score = meteor_metric.compute(predictions=generated_answers, references=ground_truth_answers)

# Calculate ROUGE score
rouge_score = rouge_metric.compute(predictions=generated_answers, references=ground_truth_answers)

print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score)
print("ROUGE Score:", rouge_score) 

/home/erap/.local/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/erap/.local/lib/python3.9/site-packages/datasets/load.py:756: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from t

BLEU Score: {'bleu': 0.6977587666643851, 'precisions': [0.87, 0.7474747474747475, 0.6428571428571429, 0.5670103092783505], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 100, 'reference_length': 100}
METEOR Score: {'meteor': 0.74504464547986}
ROUGE Score: {'rouge1': AggregateScore(low=Score(precision=0.972975, recall=0.9403249999999999, fmeasure=0.9469916666666668), mid=Score(precision=0.988, recall=0.97, fmeasure=0.971), high=Score(precision=1.0, recall=0.9925, fmeasure=0.9886666666666666)), 'rouge2': AggregateScore(low=Score(precision=0.5199583333333334, recall=0.5199875, fmeasure=0.5158124999999999), mid=Score(precision=0.6133333333333334, recall=0.6108333333333333, fmeasure=0.6033333333333334), high=Score(precision=0.6933333333333335, recall=0.695, fmeasure=0.685875)), 'rougeL': AggregateScore(low=Score(precision=0.971, recall=0.9399958333333333, fmeasure=0.9469916666666668), mid=Score(precision=0.988, recall=0.9688333333333334, fmeasure=0.9696666666666667), hig

In [29]:
# trial implementation

# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")
# model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")

# context = r"""
# World number one Novak Djokovic says he is hoping for a "positive decision" to allow him 
# to play at Indian Wells and the Miami Open next month. The United States has extended 
# its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination 
# will be required to enter the country until at least 10 April, but the Serbian has previously 
# said he is unvaccinated. The 35-year-old has applied for special permission to enter the country. 
# Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar 
# outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to 
# the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title 
# and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

# inputs = tokenizer(context, return_tensors="pt")
# outputs = model.generate(**inputs, max_length=100)
# question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
# question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
# question, answer = question_answer.split(tokenizer.sep_token)

# print("question:", question)
# print("answer:", answer)


question:  What is the best title for the passage?
answer:  Djokovic's application for special permission to enter the United States
